In [30]:
# Load all the packages needed for analyses
import sys
import numpy as np
import scipy.io as sio
from numba import jit
import matplotlib.pyplot as plt

# Set up Hopf as our model 
import WholeBrain.Models.supHopf as Hopf
from WholeBrain.simulate_SimOnly import Tmaxneuronal
Hopf.initialValue = 0.1
neuronalModel = Hopf

# Set up our integrator
import WholeBrain.Integrator_EulerMaruyama as myIntegrator
integrator = myIntegrator
integrator.neuronalModel = neuronalModel
integrator.verbose = False
integrator.clamping = False

# Set up the integration parameters
dt = 5e-5
# tmax is equal to the number of timepoints: 193
tmax= 193
ds = 1e-4
Tmaxneuronal = int((tmax+dt))

import WholeBrain.simulate_SimOnly as simulateBOLD
simulateBOLD.warmUp = True
simulateBOLD.integrator = integrator
simulateBOLD.warmUpFactor = 606./2000.

# Set up the code to obtain the variables we want to maximize similarity to empirical FC
import WholeBrain.Observables.FC as FC
#import WholeBrain.Observables.swFCD as swFCD
import WholeBrain.Observables.phFCD as phFCD
import WholeBrain.Optimizers.ParmSeep as ParmSeep
ParmSeep.simulateBOLD = simulateBOLD
ParmSeep.integrator = integrator
ParmSeep.verbose = True

# set BOLD filter settings
import WholeBrain.Utils.filteredPowerSpectralDensity as filtPowSpectr
import WholeBrain.BOLDFilters as BOLDfilters

# These filters are applied in the filtPowSpectr function that we use to extract the intrinsic frequencies of each region.
# They are also applied to process the FC and swFCD and phFCD, but you can set the corresponding parameter to False later on. 0.04-0.07 Hz common to extract intrinsic frequencies
BOLDfilters.flp = 0.04
BOLDfilters.fhi = 0.07
BOLDfilters.TR = 3.0


# Get the list of names of all regions in the AAL atlas. This is needed to get the right indices, to then filter the FC
import csv
# This is a sublist of labels of the cortical regions that were included in the paper by Demirtas et al. - AAL atlas (78 regions, excluding infratentorial and deep)
with open ('/home/riccardo/ADNI_Hopf/Utils/aal_regions_included.csv', newline='') as f:
    new_reader = csv.reader(f)
    included_regions = list(new_reader)
f.close()

# Get the AAL atlas labels
import nilearn.datasets as datasets
aal = datasets.fetch_atlas_aal()
labels = np.array(aal.labels)
# create an array with the indices of each label (note that these are not the label number from the nifti image)
indices = np.array([i for i in enumerate(labels)])
SC_regions_index = np.isin(labels, included_regions)
# filter the indices that we want based on the position so to have a final SC matrix only for the regions we considered.
SC_78_regions_aal_atlas = indices[SC_regions_index]
filter_SC = np.array([int(i) for i in SC_78_regions_aal_atlas[:,0]])

# Set file path for SC matrix
x_path = '/home/riccardo/ADNI_Hopf/Utils/'
# Load structural connectivity matrix and use it as parameter in Hopf model
xfile = 'SCmatrices88healthy.mat' 
M = sio.loadmat(x_path + xfile); 
mat = M['SCmatrices']
# averaging the SC among subjects
mat0 = np.mean(mat,axis = 0)
# Filter the SC to have just the 78 regions we considered
x_mat0 = mat0[filter_SC]
new_mat0 = x_mat0.T[filter_SC]
# Prevent full synchronization of the model
SCnorm = new_mat0 * 0.2 / new_mat0.max() 
np.fill_diagonal(SCnorm,0)
print('SCnorm.shape={}'.format(new_mat0.shape))    
Hopf.setParms({'SC':SCnorm})

# ------------------------------------------------
# Retrieve the data for all subjects 
# ------------------------------------------------
timeseries = np.load('/home/riccardo/ADNI_Hopf/Results/timeseries_all.npy')
nsubjects, nNodes, Tmax = timeseries.shape
all_fMRI = {s: d for s,d in enumerate(timeseries)} 
# Since we aleardy filtered the data in the previous step from Nilearn, we aren't going to filter them again. Otherwise, a possible alternative, could be to add another
# BOLDfilters to actually re-set the filters after the f_diff was extracted and before the call to the simulation.
distanceSettings = {'FC': (FC, False), 'phFCD': (phFCD, True)}

simulateBOLD.TR = 3.  # Recording interval: 1 sample every 3 seconds
simulateBOLD.dt = 0.1 * simulateBOLD.TR / 2.
simulateBOLD.Tmax = Tmax  # This is the length, in seconds
simulateBOLD.dtt = simulateBOLD.TR  # We are not using milliseconds
simulateBOLD.t_min = 10 * simulateBOLD.TR
# simulateBOLD.recomputeTmaxneuronal() <- do not update Tmaxneuronal this way!
# simulateBOLD.warmUpFactor = 6.
simulateBOLD.Tmaxneuronal = (Tmax-1) * simulateBOLD.TR + 30
integrator.ds = simulateBOLD.TR  # record every TR millisecond

# Hopf.beta = 0.01
f_diff = filtPowSpectr.filtPowSpetraMultipleSubjects(timeseries, TR=3.)  # should be baseline_group_ts .. or baseling_group[0].reshape((1,52,193))
f_diff[np.where(f_diff == 0)] = np.mean(f_diff[np.where(f_diff != 0)])  # f_diff(find(f_diff==0))=mean(f_diff(find(f_diff~=0)))

Hopf.omega = 2 * np.pi * f_diff

print("ADHopf Setup done!")


base_a_value = -0.02
warmUp = True
warmUpFactor = 10.

def computeSubjectSimulation():
    # integrator.neuronalModel.SC = C
    # integrator.initBookkeeping(N, Tmaxneuronal)
    if warmUp:
        currObsVars = integrator.warmUpAndSimulate(dt, Tmaxneuronal, TWarmUp=Tmaxneuronal/warmUpFactor)
    else:
        currObsVars = integrator.simulate(dt, Tmaxneuronal)
    # currObsVars = integrator.returnBookkeeping()  # curr_xn, curr_rn
    neuro_act = currObsVars[:,1,:]  # curr_rn
    return neuro_act
    

def loadXBurden(condition):
    # ------------------- load and stack the different wm burdens
    wm_hc = np.load('/home/riccardo/ADNI_Hopf/Results/wmh_volumes_HC.npy')
    wm_mci = np.load('/home/riccardo/ADNI_Hopf/Results/wmh_volumes_MCI.npy')
    wm_overall = np.load('/home/riccardo/ADNI_Hopf/Results/wmh_volumes_ALL.npy')
    # ------------------- load the specific subject wm
    if condition == 'hc':
        wmBurden = wm_hc
    elif condition == 'mci':
        wmBurden = wm_mci
    elif condition == 'all':
        wmBurden = wm_overall
    # ------------------- normalize and return
    # wmBurdenNorm = (wmBurden - np.min(wmBurden))/np.ptp(wmBurden)  # Normalize each individual in [0,1]
    wmBurdenNorm = (wmBurden - np.min(wm_overall))/np.ptp(wm_overall)  # Normalize the whole group in [0,1]
    return wmBurdenNorm

# ------------ load wm burden
conditionToStudy='all' #lower case, can be hc or mci or all
wmBurden = loadXBurden(conditionToStudy)
mode = 'homogeneous'  # homogeneous/heterogeneous

# Note that this homogeneous is intended as homogeneous inside the same patient, so all regions of one patient have the same wmBurden, but different patients have different wmBurdens.
# Heterogenous, instead, means that different regions in the same patient have different wmBurdens
if mode == 'homogeneous':
    #avgwm = np.average(wmBurden)
    wmBurden = np.array([np.ones([nNodes]) * wmBurden[i] for i in range(len(wmBurden))])
    


SCnorm.shape=(78, 78)
filtPowSpetraMultipleSubjects: subject 0 (of 33)
filtPowSpetraMultipleSubjects: subject 1 (of 33)
filtPowSpetraMultipleSubjects: subject 2 (of 33)
filtPowSpetraMultipleSubjects: subject 3 (of 33)
filtPowSpetraMultipleSubjects: subject 4 (of 33)
filtPowSpetraMultipleSubjects: subject 5 (of 33)
filtPowSpetraMultipleSubjects: subject 6 (of 33)
filtPowSpetraMultipleSubjects: subject 7 (of 33)
filtPowSpetraMultipleSubjects: subject 8 (of 33)
filtPowSpetraMultipleSubjects: subject 9 (of 33)
filtPowSpetraMultipleSubjects: subject 10 (of 33)
filtPowSpetraMultipleSubjects: subject 11 (of 33)
filtPowSpetraMultipleSubjects: subject 12 (of 33)
filtPowSpetraMultipleSubjects: subject 13 (of 33)
filtPowSpetraMultipleSubjects: subject 14 (of 33)
filtPowSpetraMultipleSubjects: subject 15 (of 33)
filtPowSpetraMultipleSubjects: subject 16 (of 33)
filtPowSpetraMultipleSubjects: subject 17 (of 33)
filtPowSpetraMultipleSubjects: subject 18 (of 33)
filtPowSpetraMultipleSubjects: subject

In [31]:
outFilePath = '/home/riccardo/ADNI_Hopf/Results/Prove'

import matplotlib.pyplot as plt
import WholeBrain.Utils.plotFitting as plotFitting


# =====================================================================
# =====================================================================
#                      Single Subject Pipeline:  L2L
# =====================================================================
# =====================================================================
def setupFunc(parms):
    wmW = parms['scaling']
    # wmParms = [{'a': base_a_value + wmW * wmBurden} for wmW in wmWs]
    neuronalModel.setParms({'a': base_a_value + wmW * wmBurden})


N = nNodes
NumTrials = 5
def fittingPipelineL2L(all_fMRI,
                       distanceSettings,  # This is a dictionary of {name: (distance module, apply filters bool)}
                       selectedObservable):
    from collections import namedtuple
    from l2l.utils.experiment import Experiment
    from l2l.optimizers.gridsearch import GridSearchOptimizer, GridSearchParameters
    import WholeBrain.Optimizers.L2LOptimizee as WBOptimizee
    from WholeBrain.Optimizers.preprocessSignal import processEmpiricalSubjects  # processBOLDSignals

    # Model Simulations
    # ------------------------------------------
    # Now, optimize all alpha (B), beta (Z) values: determine optimal (B,Z) to work with
    print("\n\n###################################################################")
    print("# Fitting (scaling)")
    print("###################################################################\n")
    experiment = Experiment(root_dir_path='Data_Produced')
    name = 'L2L-ADHopf-Prepro'
    traj, _ = experiment.prepare_experiment(name=name, log_stdout=True, multiprocessing=False)

    # Setup the WhileBrain optimizee
    WBOptimizee.neuronalModel = neuronalModel
    WBOptimizee.integrator = integrator
    WBOptimizee.simulateBOLD = simulateBOLD

    # for k in list(distanceSettings):
    #     if k != selectedObservable:
    #         del distanceSettings[k]
    # distanceSettings = {'swFCD': (swFCD, True)}
    # del distanceSettings['FC'], distanceSettings['swFCD'] #, distanceSettings['phFCD']
    WBOptimizee.measure = distanceSettings[selectedObservable][0]  # Measure to use to compute the error
    WBOptimizee.applyFilters = distanceSettings[selectedObservable][1]  # Whether to apply filters to the resulting signal or not
    outEmpFileName = outFilePath + '/fNeuro_emp_L2L.mat'
    WBOptimizee.processedEmp = processEmpiricalSubjects(all_fMRI,
                                                        distanceSettings,
                                                        outEmpFileName)[selectedObservable]  # reference values (e.g., empirical) to compare to.
    WBOptimizee.N = N  # Number of regions in the parcellation
    WBOptimizee.trials = NumTrials  # Number of trials to try
    optimizee_parameters = namedtuple('OptimizeeParameters', [])

    filePattern = outFilePath + ('/fitting_{}_L2L.mat' if mode != 'homogeneous' else '/fitting_{}_homogeneous-L2L.mat')
    optimizee = WBOptimizee.WholeBrainOptimizee(traj, {'scaling': (-0.1, 0.1)}, setupFunc=setupFunc, outFilenamePattern=filePattern)

    # =================== Test for debug only
    # traj.individual = sdict(optimizee.create_individual())
    # testing_error = optimizee.simulate(traj)
    # print("Testing error is %s", testing_error)
    # =================== end Test

    # Setup the GridSearchOptimizer
    optimizer_parameters = GridSearchParameters(param_grid={
        'scaling': (-0.1, 0.1, int((0.1+0.05-(-0.1))/0.05))
    })
    optimizer = GridSearchOptimizer(traj,
                                    optimizee_create_individual=optimizee.create_individual,
                                    optimizee_fitness_weights=(-1.,),  # minimize!
                                    parameters=optimizer_parameters)

    experiment.run_experiment(optimizee=optimizee,
                              optimizee_parameters=optimizee_parameters,
                              optimizer=optimizer,
                              optimizer_parameters=optimizer_parameters)
    experiment.end_experiment(optimizer)
    print(f"best: scaling={experiment.optimizer.best_individual['scaling']}")
    return {'subject': subjectName, 'value': experiment.optimizer.best_fitness, 'parms': experiment.optimizer.best_individual['scaling']}



subj_list = np.load('/home/riccardo/ADNI_Hopf/Results/subject_list_timeseries_all.npy')

subjectName = ''
warmUp = True
warmUpFactor = 10.

neuronalModel.setParms({'we': 2.9})
selectedObservable = 'FC'
# tauStart = 0.5
# tauWs = np.arange(tauStart, tauEnd + tauStep, tauStep)
optimal = fittingPipelineL2L(all_fMRI, distanceSettings, selectedObservable)
# ------- Save result
sio.savemat(outFilePath + f'/fittingResult-scaling-{mode}-{selectedObservable}.mat', optimal)


# def fittingPipeline_heterogeneous(all_fMRI, wmBurden, wmWs):
#     best_parameters_dict = {}
#     for k, subjectName in enumerate(subj_list):
#         subj_fMRI = {k:all_fMRI[k]}
#         wmBurden_subj = wmBurden[k]
#         best_parameters = fittingPipeline_homogeneous(subj_fMRI=subj_fMRI, distanceSettings=distanceSettings, subjectName=subjectName, wms=wmWs, wmBurden = wmBurden_subj)
#         best_parameters_dict[subjectName] = best_parameters
#     return best_parameters_dict



###################################################################
# Fitting (scaling)
###################################################################

All output logs can be found in directory  /home/riccardo/ADNI_Hopf/Code/Hopf_simulation/Python/HopfGustavoPatow_new/WholeBrain/Data_Produced/L2L-ADHopf-Prepro/logs
JUBE parameters used: {'submit_cmd': 'sbatch', 'job_file': 'job.run', 'nodes': '1', 'walltime': '01:00:00', 'ppn': '1', 'cpu_pp': '1', 'threads_pp': '4', 'mail_mode': 'ALL', 'err_file': 'stderr', 'out_file': 'stdout', 'tasks_per_job': '1', 'exec': 'python3 /home/riccardo/ADNI_Hopf/Code/Hopf_simulation/Python/HopfGustavoPatow_new/WholeBrain/Data_Produced/L2L-ADHopf-Prepro/simulation/run_files/run_optimizee.py', 'ready_file': '/home/riccardo/ADNI_Hopf/Code/Hopf_simulation/Python/HopfGustavoPatow_new/WholeBrain/Data_Produced/L2L-ADHopf-Prepro/ready_files/ready_w_', 'work_path': '/home/riccardo/ADNI_Hopf/Code/Hopf_simulation/Python/HopfGustavoPatow_new/WholeBrain/Data_Pro

ValueError: unable to broadcast argument 3 to output array
File "/home/riccardo/ADNI_Hopf/Code/Hopf_simulation/Python/HopfGustavoPatow_new/WholeBrain/WholeBrain/Integrator_EulerMaruyama.py", line 1, 